In [1]:
import os,sys
sys.path.append('../..')
sys.path.append('../../../gempy-1.16/')
import sandbox.sandbox as sb
#New daniel 
from sb import KinectV2

import gempy as gp
#import gempy_sandbox a   s gpsb
import numpy as np
#from PIL import Image, ImageDraw
import pickle

../..\sandbox\sandbox.py:8: UserWarning: Freenect is not installed. if you are using the Kinect Version 2 on a windows machine, use the KinectV2 class!
  warn('Freenect is not installed. if you are using the Kinect Version 2 on a windows machine, use the KinectV2 class!')
../../../gempy-1.16\gempy\plotting\visualization.py:31: UserWarning: Vtk package is not installed. No vtk visualization available.
  warnings.warn('Vtk package is not installed. No vtk visualization available.')
../../../gempy-1.16\gempy\plotting\visualization.py:36: UserWarning: Steno 3D package is not installed. No 3D online visualization available.
  warnings.warn('Steno 3D package is not installed. No 3D online visualization available.')
../../../gempy-1.16\gempy\data_management.py:34: UserWarning: qgrid package is not installed. No interactive dataframes available.
  warnings.warn('qgrid package is not installed. No interactive dataframes available.')
../../../gempy-1.16\gempy\posterior_analysis.py:24: UserWarnin

Version of Kinect using (1 or 2):2


### Set up the sandbox 

First we activate the kinect

In [2]:
kinect = sb.KinectV2()
kinect.sigma_gauss=2
kinect.n_frames=3


Now we can set up the projector settings:

In [3]:
projector = sb.Projector(resolution = (1280,800), work_directory='./temp',
                        refresh=100, input_rescale=False)

Next is to pass the calibration---kinect-projector---to the projector class to be able to project just on the sandbox.

In [4]:
calibration_filename = "sandbox_v2.dat" 
calibration = sb.Calibration(projector)
calibration.load(calibration_file=calibration_filename)
projector.set_calibration(calibration)


In [5]:
projector.start_stream()   

Also we need to set up the general options of Matplotlib (the library to create the models). Here we are passing the colors of GemPy simply for consistency.

In [14]:
plot_im = sb.Plot(
    calibration=calibration,
    cmap=gp.plotting.colors.cmap,
    norm=gp.plotting.colors.norm
    )

### Setup the model gempy model

Now we need to generate the map that we want to poject. In the case of a geological map we will use GemPy.

First we need to load the underlaying 3D model. We can find some examples in the GemPy library

In [15]:
geo_model = gp.read_pickle(r'./../../../gempy-1.16/notebooks/examples/Gempy Modelle Lisa/lisa-7.pickle')

We want to scale the sandbox next to the model extent. We can see a gempy model extent by

In [16]:
geo_model._geo_data.extent

array([   0, 2000,    0, 2000,    0, 1600])

You can just pass those values. However the maximum high of the sandbox may to be to high. If that is the case you can just translate the whole extent up and down by changing the z values (i.e. the last two components of the extent)

### Create sandbox grid following the right scale sandbox-model

Now we need to connect the cloud of points recorded by the kinect to the scale of geological model

In [17]:
scale=sb.Scale(extent=[   0, 2000,    0, 2000,    500, 2500], calibration=calibration) #initialize scale object to handle the scaling of the model in the andbox
scale.calculate_scales()
grid = sb.Grid(calibration=calibration,scale=scale)

Aspect ratio of the model is fixed in XY
Model size is limited by Y dimension
scale in Model units/ mm (X,Y,Z): [2.5478260869565217, 3.333333333333333, 4.0]
the shown extent is [0.0, 2539.1304347826085, 0.0, 1991.3043478260868] 


### Create the GeoMap module for the Sandbox

To increase the flexibility of the sandbox and generalize the hyperthreading (i.e. getting the sandbox running without blocking python), we have create a modules systems. This modules are classes that encapsulate the specific functionality for each type of map that we want to render. Lets create the geological map module.

The work_directory in this class is where we write the image of the geological map. Therefore is **important that it is pointing to same place as the projector class**---which renders the image in the projector.

In [18]:
geo_module = sb.GeoMapModule(geo_model, grid, plot_im, work_directory=projector.work_directory)

This box is not necessary but it is good to test that the model gets computed without any error. If nothing is broken, you are good to go:

In [19]:
geo_module.render_geo_map(*geo_module.compute_model(kinect.get_filtered_frame()))

### Hyperthreading

Hyperthreading is important not to block the python script once the sandbox is activated. Later we will show some things we can change while the model is running.

In [20]:
# This will create the threading object which contains the functionality related to the thread itself
s_thread = sb.SandboxThread(geo_module, kinect, projector)

Running the thread we will initialize the real-time updating in the sandbox. If you have made it up to here you have done most of the work.

In [21]:
s_thread.run()
with s_thread.lock:
    kinect.sigma_gauss=10
    kinect.n_frames=10

Exception in thread Thread-57:
Traceback (most recent call last):
  File "C:\Users\Admin\Miniconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Admin\Miniconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "../..\sandbox\sandbox.py", line 1384, in loop
    self.projector.show()
  File "../..\sandbox\sandbox.py", line 715, in show
    os.replace(os.path.join(self.work_directory, 'output_temp.png'), os.path.join(self.work_directory, 'output.png')) #workaround to supress artifacts
PermissionError: [WinError 5] Access is denied: './temp\\output_temp.png' -> './temp\\output.png'



**Congratulations your Sandbox (should be) is running!!**
<hr style='border-width: 6px'>


Now if you want to stop the thread you can do it by calling next cell. (By default we leave it commented just delete the Hashtag)

In [ ]:
s_thread.kill()

## Real time-manipulation

Now that the hyperthreading is on, we can do couple of cool things. Most of the code from here on is written under the 

```python
with s_thread.lock:
    ...
```

This is to guarantee that one thread is not messing with the other. Lets start to see what good stuff can we do

#### Show/hide topography

In [ ]:
with s_thread.lock:
    geo_module.plot_topography = True

#### Show/hide faults

In [ ]:
with s_thread.lock:
    geo_module.plot_faults = True

#### Moving the extent:

In [ ]:
with s_thread.lock:
    scale.extent = [0, 2000,    0, 2000,   1000, 2500]
    scale.calculate_scales()

#### Save geological/topology map
(Okay these two work sometimes work better if you kill the thread first)

In [ ]:
with s_thread.lock:
    geo_module.export_geological_map(kinect.get_filtered_frame(),
                                     'geological_map.pdf')

In [ ]:
with s_thread.lock:
    geo_module.export_topographic_map('topographic_map.pdf')

#### change contour levels 

In [ ]:
with s_thread.lock:
    geo_module.create_sub_contours(0,2000,step=50)

### Loading GemPy models and use GemPy functionality

This is a big one. Since the  thread is free we can still use gempy. The main thing to keep in ming is **not to use the same name for the variable when you load a new model**. Otherwise you risk to mixing which variable is which model and where it is.

For example we can visualize the model by reading it again:

In [ ]:
geo_model_grab = gp.read_pickle(r'./../../../gempy/notebooks/examples/Models_Lisa/lisa-7.pickle')

Computing it:

In [ ]:
with s_thread.lock:
    l, f = gp.compute_model(geo_model_grab)


And plotting it:

In [ ]:
with s_thread.lock:
    gp.plotting.plot_section(geo_model_grab._geo_data, l[0], 25)

### Finally we can swap models as well!

In the same fashion that we have imported the same model to visualize the section, we can load another model and swap it. **Be careful about the extent** if the second model has a different exntent you may be projecting the wrong area:

In [ ]:
with s_thread.lock:
    geo_model_swap= gp.read_pickle(r'./../../../gempy-1.16/notebooks/examples/Gempy Modelle Lisa/lisa-7.pickle')

In [ ]:
with s_thread.lock:
    # The names choice is aweful. In new iteration we will use better the alphabet
    geo_module.geo_model = geo_model_swap

In [ ]:
with s_thread.lock:
    kinect.sigma_gauss=2
    kinect.n_frames=6

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with s_thread.lock:
    # Create buttoms:
    buttoms = []

    # Create callback
    def change_model(x):
        with s_thread.lock:

            # Reset buttoms icon
            for i in buttoms:
                i.icon = ''
            x.icon ='check'

            model = x.description[-1]
            geo_model_swap= gp.read_pickle(r'./../../../gempy-1.16/notebooks/examples/Gempy Modelle Lisa/lisa-'+model+'.pickle')
            geo_module.geo_model = geo_model_swap

    for m in (['Model 1' ,'Model 2', 'Model 3','Model 5', 'Model 6','Model 7','Model 8', 'Model 9']):

        w = widgets.Button(
                            description=m,
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip=m,
                            )
        w.on_click(change_model)
        buttoms.append(w)


In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Click on Models to change the underlying 3D model
 
1.

2.

3.

5.

6.

7.

8.

In [ ]:
widgets.HBox(buttoms[:4])

In [ ]:
widgets.HBox(buttoms[4:])

In [ ]:
def call_back(z_ext, faults, topo):
    with s_thread.lock:
        try:
            scale.extent = [0, 2000,    0, 2000,  z_ext[0], z_ext[1]]
            scale.calculate_scales()
            geo_module.plot_topography = topo
            geo_module.plot_faults = faults
        except ValueError:
            pass

In [ ]:
interact(call_back, z_ext =  widgets.IntRangeSlider(value= (500, 2500), min= -500,max= 2500, step=100, description= 'z extent'),
        faults=True, topo=True);


In [ ]:
with s_thread.lock:

    lith_block, fault_block = gp.compute_model(geo_model_swap.interp_data, get_potential_at_interfaces=False)
    #ver , sim = gp.get_surfaces(interp_data, lith_block[1], fault_block[1])

    # 2D Plot
    %matplotlib inline
    gp.plotting.plot_section(geo_data, lith_block[0], cell_number=10,
                             direction='y', plot_data=True)

In [ ]:
s_thread.pause()

In [ ]:
s_thread.kill()


In [ ]:
with s_thread.lock:
    projector.start_stream() 